<h1>Big data Competition</h1>

In [1]:
ruta=r"C:\Users\lordg\Desktop\Jacobs resourses\big_data_comp"
import pandas as pd

In [2]:
df_train=pd.read_csv(ruta+'\\train.csv')

<h2>Let's build 22 PCA mapping e.g. 1 per label.</h2>
<p>We try to reduce dimensions. Usume 42 is got number of principal vector per PCA</p>

In [3]:
from sklearn.decomposition import PCA
labels=list(df_train.Label.unique())
#labels='stand-to-sit'
models=[]

for each in labels:
    print(each)
    pca = PCA (n_components =275)
    df=df_train[df_train['Label']==each]
    files=list(df.Datafile)
    arr={}
    col_sub='EMG1,EMG2,EMG3,EMG4,Airborne,ACC_upper_X,ACC_upper_Y,ACC_upper_Z,Goniometer_X,ACC_lower_X,ACC_lower_Y,ACC_loewr_Z,Goniometer_Y,Gyro_upper_X,Gyro_upper_Y,Gyro_upper_Z,Gyro_lower_X,Gyro_lower_Y,Gyro_lower_Z'
    j=0
    for each_file in files:
        #print(each_file)
        df_explo=pd.read_csv(ruta+'\\Subjects\\'+each_file.replace('/','\\'),names=col_sub.split(','))
        #print(each_file,each)
        try:
            x=list(df_explo.values[10:3000].reshape((1, 56810))[0]) 
        except:
            #complete by repetition
            df_temp=df_explo.append(df_explo).append(df_explo).append(df_explo)
            x=list(df_temp.values[10:3000].reshape((1, 56810))[0])
        arr[j]=x
        j=j+1
    for_pca=pd.DataFrame.from_dict(arr, orient='index')
    
    print(each,'number of rows',j)
    if(each=='lay'):
        pca = PCA (n_components =16)
        
    models.append(pca.fit(for_pca))
        
    

curve-left-step
curve-left-step number of rows 299
stand-to-sit
stand-to-sit number of rows 289
curve-right-spin-Rfirst
curve-right-spin-Rfirst number of rows 300
jump-one-leg
jump-one-leg number of rows 279
lateral-shuffle-right
lateral-shuffle-right number of rows 277
curve-right-spin-Lfirst
curve-right-spin-Lfirst number of rows 301
v-cut-right-Lfirst
v-cut-right-Lfirst number of rows 300
stair-down
stair-down number of rows 278
v-cut-left-Rfirst
v-cut-left-Rfirst number of rows 280
v-cut-right-Rfirst
v-cut-right-Rfirst number of rows 280
jump-two-leg
jump-two-leg number of rows 280
sit
sit number of rows 289
stair-up
stair-up number of rows 278
curve-right-step
curve-right-step number of rows 295
sit-to-stand
sit-to-stand number of rows 289
run
run number of rows 300
v-cut-left-Lfirst
v-cut-left-Lfirst number of rows 300
stand
stand number of rows 289
curve-left-spin-Lfirst
curve-left-spin-Lfirst number of rows 280
walk
walk number of rows 300
curve-left-spin-Rfirst
curve-left-spin

<h1>Using pickle, we can save python object on our hard drive, for later use</h1>

In [4]:
#From saving the object models as a file for later usage
import pickle
with open("models_pca.file", "wb") as f:
    pickle.dump(models, f, pickle.HIGHEST_PROTOCOL)

In [5]:
import pickle
#calling from hard drive
with open("models_pca.file", "rb") as f:
    models_pca = pickle.load(f)
    # Now you can use the dump object as the original one  
    #self.some_property = dump.some_property

<h1>Using our 22 PCA, we train 22  logistic regression models. Here we use 10 fold cross validation for estimating the test error and to estimate the regularization parameter C. We also use f1_score to measure performance.</h1>
<p> each of the these model will give us an estimation of the probability of the one time series for instance to belongs or not to a particular class. For example let’s say that the MODEL(PCA_1) finds a 90%  probability that it belongs to the class 'stand', if we compare this against the second model, and we see  that it would give us 85% probability of that time series being of class 'run', then we would conclude that it belongs to the class 'stand' since 90%>85%  </p>

In [8]:
import warnings
warnings.filterwarnings('ignore')


from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import make_scorer

df_train=pd.read_csv(ruta+'\\train.csv')
import numpy as np
#Data_PCA=model[0].transform(Data)


labels=list(pd.read_csv(ruta+'\\train.csv').Label.unique())

index_label=0

#here we our 22 models.
list_models=[]

for each in labels:
    df_train=pd.read_csv(ruta+'\\train.csv')
    #df=df_train[df_train['Label']==each]
    df_train['Label'] = df_train['Label'].apply(lambda x: 1 if x==each else 0)
    l_df=list(df_train.values)
    files=list(df_train.Datafile)
    arr_trans={}
    col_sub='EMG1,EMG2,EMG3,EMG4,Airborne,ACC_upper_X,ACC_upper_Y,ACC_upper_Z,Goniometer_X,ACC_lower_X,ACC_lower_Y,ACC_loewr_Z,Goniometer_Y,Gyro_upper_X,Gyro_upper_Y,Gyro_upper_Z,Gyro_lower_X,Gyro_lower_Y,Gyro_lower_Z'
    j=0
    dict_val_X,dict_val_y={},{}
    for each_file in files:
        #print(each_file)
        df_explo=pd.read_csv(ruta+'\\Subjects\\'+each_file.replace('/','\\'),names=col_sub.split(','))
        #print(each_file,each)
        try:
            x=np.array(df_explo.values[10:3000].reshape((1, 56810))[0]) 
        except:
            #complete by repetition
            df_temp=df_explo.append(df_explo).append(df_explo).append(df_explo)
            x=np.array(df_temp.values[10:3000].reshape((1, 56810))[0])
        dict_val_X[j]=models_pca[index_label].transform(x.reshape(1, -1))[0]
        dict_val_y[j]=l_df[j][2]
        j=j+1
        
    X=np.array(list(dict_val_X.values()))
    y=np.array(list(dict_val_y.values()))
    f1_score_weighted = make_scorer(f1_score, average='weighted')
    CV_LR = LogisticRegressionCV(cv=10,  Cs=[0.1,0.5, 1,1.5, 5,10,50, 100],scoring=f1_score_weighted)
    #Este el objeto que nos interesa, este el modelo entranado.
    cv_classifier_LR= CV_LR.fit(X, y)
    list_models.append(cv_classifier_LR)
    print('classifier for ',each, ' ready. X_shape:', X.shape)
    index_label += 1

    

classifier for  curve-left-step  ready. X_shape: (6401, 275)
classifier for  stand-to-sit  ready. X_shape: (6401, 275)
classifier for  curve-right-spin-Rfirst  ready. X_shape: (6401, 275)
classifier for  jump-one-leg  ready. X_shape: (6401, 275)
classifier for  lateral-shuffle-right  ready. X_shape: (6401, 275)
classifier for  curve-right-spin-Lfirst  ready. X_shape: (6401, 275)
classifier for  v-cut-right-Lfirst  ready. X_shape: (6401, 275)
classifier for  stair-down  ready. X_shape: (6401, 275)
classifier for  v-cut-left-Rfirst  ready. X_shape: (6401, 275)
classifier for  v-cut-right-Rfirst  ready. X_shape: (6401, 275)
classifier for  jump-two-leg  ready. X_shape: (6401, 275)
classifier for  sit  ready. X_shape: (6401, 275)
classifier for  stair-up  ready. X_shape: (6401, 275)
classifier for  curve-right-step  ready. X_shape: (6401, 275)
classifier for  sit-to-stand  ready. X_shape: (6401, 275)
classifier for  run  ready. X_shape: (6401, 275)
classifier for  v-cut-left-Lfirst  ready.

In [9]:
#From saving the object models as a file for later usage
import pickle
with open("list_models.file", "wb") as f:
    pickle.dump(list_models, f, pickle.HIGHEST_PROTOCOL)

In [10]:

import pickle
with open("list_models.file", "rb") as f:
    list_models = pickle.load(f)
    # Now you can use the dump object as the original one  
    #self.some_property = dump.some_property

<h1>Finally we make predictions</h1>

In [11]:
import numpy as np
import operator

labels=list(pd.read_csv(ruta+'\\train.csv').Label.unique())

real_pred=[]
df_test=pd.read_csv(ruta+'\\train.csv')
files=list(df_test.Datafile)
arr={}
#ln_labels=list(df_test.Label)
for each_data in files:
    df_explo=pd.read_csv(ruta+'\\Subjects\\'+each_data.replace('/','\\'))
    try:
        x=np.array(df_explo.values[10:3000].reshape((1, 56810))[0]) 
    except:
        #complete by repetition
        df_temp=df_explo.append(df_explo).append(df_explo).append(df_explo)
        x=np.array(df_temp.values[10:3000].reshape((1, 56810))[0])
    
    index_label=0
    arr={}
    for each_label in labels:
        predi=models_pca[index_label].transform(x.reshape(1, -1))[0]
        XX=list_models[index_label].predict_proba(predi.reshape(1, -1))
        arr[index_label]=[XX[0][0]]
        #larr[index_label]=[ln_labels[index_label]]
        index_label += 1
    #here we select the maximum probality of the array of class predictions probalities
    
    real_pred.append(arr)
    
    #real_pred.append(labels[max(arr.items(), key=operator.itemgetter(1))[0]])

<h1>Let us reconstruct the test data for submission.</h1>

In [ ]:
#dicrr={}

#Subject_c=list(df_test.Subject.values)
#dicrr['Subject']=Subject_c

#Datafile_c=list(df_test.Datafile.values)
#dicrr['Datafile']=Datafile_c

#dicrr['Label_pre']=real_pred

#Subject_c=list(df_test.Label.values)
#dicrr['Label']=Subject_c

In [12]:
#real_pred

X_train_dic={}
y_train_dic={}



In [13]:
X_train_dic['Label_predicted_array']=real_pred

fuck={}
nn=0
al=[]
for each in real_pred:
    #print(list(each.values()))
    #nn=0
    for cada in list(each.values()):
        al.append(cada[0])
    fuck[nn]=al
    nn=nn+1
    al=[]
    
#data=np.array(fuck)
df = pd.DataFrame(fuck)  
#fuck
#fuck[1]
r_df=df.T
r_df.columns=labels

classes=list(df_test.Label.values)
y_train_dic['Target']=classes
#dicrr['real']=real_pred
#X_train_dic['Label_predicted_array']=real_pred
#Subject,Datafile,Label

data_X_train=r_df
data_y_train=pd.DataFrame.from_dict(y_train_dic)   




In [22]:

#splitting 75% train, 25% test
from sklearn.model_selection import train_test_split


#From saving the object models as a file for later usage
import pickle
with open("data_X_train.file", "wb") as f:
    pickle.dump(data_X_train, f, pickle.HIGHEST_PROTOCOL)
    
with open("data_y_train.file", "wb") as f:
    pickle.dump(data_y_train, f, pickle.HIGHEST_PROTOCOL)

    
with open("data_X_train.file", "rb") as f:
    data_X_train = pickle.load(f)
    # Now you can use the dump object as the original one  
    #self.some_property = dump.some_property

with open("data_y_train.file", "rb") as f:
    data_y_train = pickle.load(f)
    # Now you can use the dump object as the original one  
    #self.some_property = dump.some_property
    
#random_state is the seed of the random selection, use the same to ensure reproducibility.
X_train,X_test,y_train,y_test = train_test_split(data_X_train,data_y_train,test_size=0.25,random_state=0)  

In [14]:
#we import the class tree from sklearn

from sklearn import tree

#Create the object model.
tree_model = tree.DecisionTreeClassifier()
#Fit the model.
tree_model = tree_model.fit(X_train, y_train)

In [ ]:
#This class would allow us to draw the decision Tree.
#from io import StringIO
#import pydot 
#import graphviz 
#dot_data = tree.export_graphviz(tree_model, out_file=None,
#                                feature_names=X_train.columns, 
#                                class_names=classes, 
#                                filled=True,
#                                special_characters=True)  
#graph = graphviz.Source(dot_data)  
#Show inline in jupyter notebook
#graph 

#For saving as pdf
#dot_data = StringIO() 
#tree.export_graphviz(tree_model, 
#                     out_file=dot_data,
#                     feature_names=X_train.columns, 
#                     class_names=classes, 
#                     filled=True,
#                     special_characters=True) 
#graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
#graph[0].write_pdf("tree_challange.pdf") 

In [15]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

#Here create the martrix of prediction over the test sets.
y_pred_test=tree_model.predict(X_test)

#Here create the martrix of prediction over the train sets.
y_pred_train=tree_model.predict(X_train)



# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred_test)
np.set_printoptions(precision=2)



In [16]:
#Metrics definition.
def accuracy(confusion_matrix):
    sum_diagonal=sum(np.diagonal(confusion_matrix))
    total=sum(sum(confusion_matrix))
    return (sum_diagonal/total)

def precision(label, confusion_matrix):
    col = confusion_matrix[:, label]
    return confusion_matrix[label, label] / col.sum()
    
def recall(label, confusion_matrix):
    row = confusion_matrix[label, :]
    return confusion_matrix[label, label] / row.sum()

In [17]:
print("Overall TEST accuracy")
print("----->",accuracy(cnf_matrix))
print("------------------------------------------------")

print("Overall TRAIN accuracy")
print("----->",accuracy(confusion_matrix(y_train, y_pred_train)))
print("------------------------------------------------")



Overall TEST accuracy
-----> 0.7601499063085572
------------------------------------------------
Overall TRAIN accuracy
-----> 1.0
------------------------------------------------


<h1>let us try random forest</h1>


In [21]:
from sklearn.ensemble import GradientBoostingClassifier 


#instate the object model.
boosting_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.45, max_depth=7, random_state=0)
#Fit the model.
boosting_model = boosting_model.fit(X_train, y_train)

#Here create the martrix of prediction over the test sets.
y_pred_test=boosting_model.predict(X_test)
#Here create the martrix of prediction over the train sets.
y_pred_train=boosting_model.predict(X_train)

cnf_boosting_test_matrix = confusion_matrix(y_test, y_pred_test)
cnf_boosting_train_matrix = confusion_matrix(y_train, y_pred_train)

print("boosting:\n")
print("Overall TEST accuracy")
print("----->",accuracy(cnf_boosting_test_matrix))
print("------------------------------------------------")

print("Overall TRAIN accuracy")
print("----->",accuracy(cnf_boosting_train_matrix))
print("------------------------------------------------")



boosting:

Overall TEST accuracy
-----> 0.8107432854465959
------------------------------------------------
Overall TRAIN accuracy
-----> 0.9995833333333334
------------------------------------------------


In [ ]:
dicrr={}

Subject_c=list(df_test.Subject.values)
dicrr['Subject']=Subject_c

Datafile_c=list(df_test.Datafile.values)
dicrr['Datafile']=Datafile_c

dicrr['Label_pre']=y_pred_test

Subject_c=list(df_test.Label.values)
dicrr['Label']=Subject_c

In [ ]:
data_frame_out=pd.DataFrame.from_dict(dicrr)

In [ ]:
data_frame_out

In [ ]:
#data_frame_out.to_csv(index=False,path_or_buf=ruta+'test_03_04.csv')

<h1>Example diagram</h1>

aquiiiii